In [6]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "<h1>Colab連線成功</h1>"

# Define the port variable
port = 5000 # You can change this to any available port number

if __name__ == '__main__':
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [7]:
!pip install pyngrok


In [8]:
from pyngrok import ngrok
from google.colab import userdata

ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))


In [9]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def home():
    return "<h1>Colab連線成功</h1>"

app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [11]:

!pip install Flask pyngrok line-bot-sdk requests # openai
!pip install google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.3 MB/s eta 0:00:00


In [12]:

from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
port=5051

In [13]:

ngrok.kill()

In [14]:

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(port, name="linebot_tunnel")
print(tunnel.public_url)

https://unlaboured-dillon-laboriously.ngrok-free.dev


In [ ]:

from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "<h1>Colab連線成功</h1>"

if __name__ == '__main__':
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Dec/2025 03:13:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2025 03:13:33] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:

ngrok.kill()

In [ ]:

!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [ ]:

from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [ ]:

import os
from pyngrok import ngrok

In [ ]:

ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:

result = stateful_query("簡介明新科技大學")
print(result)

In [ ]:

result2 = stateful_query("校長是誰？")
print(result2)

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)